In [1]:
import librosa
import glob
import pandas as pd

In [2]:
#repeating the process for mp3 files in train1,train2,train3,train4
#audio_files=glob.glob('./train1/*.mp3')
#audio_files=glob.glob('./train2/*.mp3')
audio_files=glob.glob('./train3/*.mp3')

In [3]:
import numpy as np
import sklearn
def spectral_properties(y: np.ndarray, fs: int,name: str) -> dict:
    spec = np.abs(np.fft.rfft(y,axis=0))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    #print(freq.shape)
    amp = spec / spec.sum()
    amp=amp.T
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
    mfcc=librosa.feature.mfcc(y, sr=sr)
    #print(mfcc.shape)
    centroid=sklearn.preprocessing.minmax_scale(spectral_centroids, axis=0)
    spectral_rolloff=librosa.feature.spectral_rolloff(y, sr=sr)[0]
    rolloff=sklearn.preprocessing.minmax_scale(spectral_rolloff, axis=0)
    zcr = librosa.feature.zero_crossing_rate(y)
    result_d = {
        'name':name,
        'mean': mean/10000,
        'sd': sd/10000,
        'median': median/10000,
        'mode': mode/10000,
        'Q25': Q25/10000,
        'Q75': Q75/10000,
        'IQR': IQR/10000,
        'skew': skew,
        'kurt': kurt,
        'mfcc':mfcc,
        'centroid':centroid,
        'rolloff':rolloff,
        'zcr':zcr
    }

    return result_d

In [ ]:
train_set=[]
i=0
for file in audio_files:
    try:
        audio,sr=librosa.load(file)
        fname=file.lstrip('./train3\\')#substituting train1,train2,train4 also.
        features=spectral_properties(audio,sr,fname)
        train_set.append(features)
        i=i+1
        print(i)
    except:
        pass;

In [5]:
len(train_set)

1541

In [9]:
train_data_all=pd.DataFrame()# creating a dataframe to have all the training data in single df.
#creating a csv file for backup extracted features table.
train1=pd.read_csv("train_1.csv",encoding="utf-8")

train1

In [ ]:
train2=pd.read_csv("train_2.csv",encoding="utf-8")

train2

In [13]:
train3=pd.read_csv("train_3.csv",encoding="utf-8")

train3

In [17]:
train4=pd.read_csv("train_4.csv",encoding="utf-8")

train4

,name,mean,sd,median,mode,Q25,Q75,IQR,skew,kurt,mfcc,centroid,rolloff,zcr
0,mple-012573.mp3,0.194367,0.191188,0.117455,0.047601,0.049431,0.251134,0.201703,6.553719,66.224619,[[-593.19666 -528.8335 -493.33853 ... -...,[0.32158485 0.34691108 0.36930127 0.36245969 0...,[0.48175182 0.52554745 0.55656934 0.55656934 0...,[[0.09375 0.14208984 0.17773438 0.17626953 ...
1,mple-012575.mp3,0.165000,0.214043,0.059911,0.013430,0.031998,0.226050,0.194052,6.670914,65.796175,[[-6.5448102e+02 -6.5181293e+02 -6.4262360e+02...,[1.00000000e+00 9.39558413e-01 8.74085338e-01 ...,[1. 0.9869961 0.9739922 0.93107932 0...,[[0.14648438 0.21679688 0.26318359 0.23046875 ...
2,mple-012581.mp3,0.244908,0.298934,0.092161,0.017292,0.035833,0.336979,0.301146,10.208999,154.471309,[[-714.1712 -714.1712 -714.1712 ... -...,[0. 0. 0. 0.67210849 0...,[0. 0. 0. 0.89450056 0...,[[0. 0. 0. 0.06298828 ...
3,mple-012587.mp3,0.344410,0.296117,0.203265,0.045127,0.096327,0.567669,0.471342,4.697479,39.254664,[[-7.8203748e+02 -7.6766479e+02 -4.9579733e+02...,[0.40674937 0.42266252 0.37717753 0.34524629 0...,[0.63272727 0.62060606 0.58060606 0.5769697 0...,[[0.13671875 0.20507812 0.22558594 0.19384766 ...
4,mple-012593.mp3,0.152568,0.090721,0.142816,0.072490,0.073457,0.210841,0.137384,5.091455,43.384484,[[-5.64965393e+02 -5.50110962e+02 -5.46663513e...,[0. 0.01112653 0.08719687 0.28987991 0...,[0. 0.04580153 0.25572519 0.71374046 0...,[[0.05761719 0.06835938 0.08154297 0.0703125 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,mple-016677.mp3,0.230208,0.253438,0.157489,0.021478,0.039930,0.293203,0.253273,7.211361,78.732812,[[-6.9004810e+02 -6.5160608e+02 -5.8421722e+02...,[0.60129067 0.59359403 0.5813135 0.54963997 0...,[0.72328767 0.64246575 0.53561644 0.50547945 0...,[[0.10791016 0.15673828 0.19921875 0.21240234 ...
1537,mple-016679.mp3,0.152535,0.187135,0.069482,0.043485,0.032335,0.191780,0.159445,5.608601,44.232227,[[-6.2704449e+02 -6.2704449e+02 -5.3794812e+02...,[0. 0.57016473 0.42988092 0.39582014 0...,[0. 0.75483871 0.55096774 0.49677419 0...,[[0. 0.05126953 0.05712891 0.07080078 ...
1538,mple-016683.mp3,0.103847,0.092569,0.081700,0.045946,0.049662,0.135303,0.085641,5.435486,42.530549,[[-8.1789355e+02 -8.1735492e+02 -7.0576343e+02...,[0.66592274 0.66784258 0.19412583 0.11970923 0...,[0.83455497 0.81256545 0.14659686 0.12879581 0...,[[0.1484375 0.27587891 0.29589844 0.30273438 ...
1539,mple-016685.mp3,0.203265,0.267158,0.064289,0.040263,0.040308,0.282155,0.241847,8.072502,89.123548,[[-7.0714960e+02 -7.0714960e+02 -7.0714960e+02...,[6.18527863e-01 6.33616443e-01 6.56448337e-01 ...,[0.80866426 0.84837545 0.86401925 0.88086643 0...,[[0.13183594 0.29003906 0.46630859 0.62646484 ...


In [19]:
train_data_all=train_data_all.append(train1)

In [20]:
train_data_all=train_data_all.append(train2)

In [21]:
train_data_all=train_data_all.append(train3)

In [22]:
train_data_all=train_data_all.append(train4)

In [23]:
len(train_data_all)

8007

In [24]:
train_data_all.to_csv("train_data_1.csv",encoding="utf-8")